In [ ]:
# !pip install neurox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 988.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install backend dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
pip install minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.2 MB/s eta 0:00:00


In [ ]:
!pip install conllu

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 929.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=9afffddd632ff0149c0b2dd8a3baad1db73414cfe47f200757572e24071ff386
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install transformers

Окончания

1 Sg -мя

2 Sg -тя

3 Sg -

1 Pl -на

2 Pl -вы

3 Pl -


In [ ]:
from transformers import AutoTokenizer

In [ ]:
from pymorphy2 import MorphAnalyzer

In [ ]:
from conllu import parse_incr, parse_tree_incr, parse

In [ ]:
import pandas as pd
import os

In [ ]:
conllu2openc = {'NOUN': 'NOUN', 'PRON': 'NPRO', 'ADJ': 'ADJF', 'DET': 'ADJF', 'VERB': 'PRTF','PROPN': 'NOUN', 'NUM':'ADJF',  \
                             'Plur':'plur', 'Sing': 'sing','Masc':'masc', 'Neut':'neut', 'Fem': 'femn', 'Anim': 'anim', 'Inan': 'inan', None:None}
pr_to_fl = {'я':'ю', 'мы':'ем', 'ты':'ешь', 'вы':'ете'}
num_to_fl = {'Sing':'ет', 'Plur':'ете', None:None}

In [ ]:
morph = MorphAnalyzer()

In [ ]:
import os
import zipfile

zf = zipfile.ZipFile("/content/drive/MyDrive/UD_converted/poly_agr.zip", "w")

zf.write('/content/drive/MyDrive/UD_converted/Biblioteka_prikluchenij_poly_agr.csv')
zf.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/UD_converted/poly_agr.zip'

In [ ]:
import shutil
shutil.make_archive('/content/drive/MyDrive/UD_converted/poly_agr', 'zip', '/content/drive/MyDrive/UD_converted/Biblioteka_prikluchenij_poly_agr.csv')

'/content/drive/MyDrive/UD_converted/poly_agr.zip'

In [ ]:
def convert_ud_file(file_path, save_path):
    data_file = open(file_path, 'r', encoding='utf-8')
    data_iter = parse_incr(data_file)
    samples = []
    for sample in data_iter:
        res = ''
        for token in sample:
            if token['feats'] is not None and token['feats'].get('Case', None) == 'Acc' and token['deprel']  == 'obj': #or ((token['upos'] in ['ADJ', 'DET', 'VERB', 'PROPN']) and sample[token['head']-1]['deprel']  == 'obj')):
                if token['lemma'] == 'я':
                  sample[token['head']-1]['form'] += 'ю'
                elif token['lemma'] == 'мы':
                  sample[token['head']-1]['form'] += 'ем'
                elif token['lemma'] == 'ты':
                  sample[token['head']-1]['form'] += 'ешь'
                elif token['lemma'] == 'вы':
                  sample[token['head']-1]['form'] += 'ете'
                elif token['feats'].get('Number', None) == 'Sing':
                  sample[token['head']-1]['form'] += 'ет'
                elif token['feats'].get('Number', None) == 'Plur':
                  sample[token['head']-1]['form'] += 'ете'
                else:
                  pass

                '''
                ('ем', 'ете', 'ют', 'ю', 'ешь', 'ет')
                anas = morph.parse(token['lemma'])
                for ana in anas:
                    if ana.tag.POS == conllu2openc[token['upos']]:
                        break
                tags = set(['nomn',conllu2openc[token['feats'].get('Number', None)], conllu2openc[token['feats'].get('Gender', None)]]) - set([None])
                new_form = ana.inflect(tags)
                if new_form is None:
                    new_form = ana.normal_form
                else:
                    new_form = new_form.word
                if token['form'] != token['form'].lower():
                    new_form = new_form.capitalize()

                res += new_form + ' '*int(token['misc'] is None)

            else:
                res += token['form'] + ' '*int(token['misc'] is None)
                '''
        #вроде нужно отдельно написать слияние форм
        samples.append(res.strip())
    data_file.close()
    df = pd.DataFrame({'text': samples})
    df.to_csv(save_path)

def convert_ud_bytes(bs, save_path):
    data_s = bs.decode('utf-8')
    data_iter = parse(data_s)
    samples = []
    for sample in data_iter:
        res = []
        mod_ids = {}
        for token in sample:
            # if token['feats'] is not None and token['feats'].get('Case', None) == 'Acc':
            #     print(token)
            res.append(token['form'])
            res.append(''+ ' '*int(token['misc'] is None))
        #     if token['feats'] is not None and token['feats'].get('Case', None) == 'Acc' and token['deprel']  == 'obj' and sample[token['head']-1]['feats'] is not None and sample[token['head']-1]['feats'].get('VerbForm', False) == 'Fin':
        #         if pr_to_fl.get(token['lemma'], False):
        #             mod_ids[(token['head']-1)*2] = pr_to_fl.get(token['lemma'], False)
        #         elif num_to_fl.get(token['feats'].get('Number', None)):
        #             mod_ids[(token['head']-1)*2] = num_to_fl.get(token['feats'].get('Number', None))
        #         else:
        #             mod_ids[(token['head']-1)*2] = num_to_fl.get('Sing')
        #             # print(sample)
        #             # print(token['feats'])
        #             # raise Exception
        #     # else:
        #     #     res += token['form'] + ' '*int(token['misc'] is None)
        # for ind, fl in mod_ids.items():
        #     res[ind] = res[ind] + fl
        samples.append(''.join(res).strip())
    # data_file.close()
    df = pd.DataFrame({'text': samples})
    with open(os.path.join(save_path, 'Biblioteka_prikluchenij_base.csv'), 'a') as f:
        f.write(df.to_csv(header=False))



In [ ]:
with open('/content/drive/MyDrive/UD/ru_syntagrus-ud-test.conllu', 'r', encoding='utf-8') as f:
    data_iter =  parse_incr(f)
    for i, s in enumerate(data_iter):
        if i == 2:
            break

In [ ]:
s

TokenList<С, одной, стороны, ,, квалифицированные, кадры, и, развитая, производственная, инфраструктура, резко, отличали, Армению, от, других, регионов, СССР, ,, где, доминировали, добывающие, отрасли, ,, а, экономика, строилась, на, поставке, сырьевых, ресурсов, ., metadata={sent_id: "2003Armeniya.xml_3", text: "С одной стороны, квалифицированные кадры и развитая производственная инфраструктура резко отличали Армению от других регионов СССР, где доминировали добывающие отрасли, а экономика строилась на поставке сырьевых ресурсов."}>

In [ ]:
res

['С',
 ' ',
 'одной',
 ' ',
 'стороны',
 '',
 ',',
 ' ',
 'квалифицированные',
 ' ',
 'кадры',
 ' ',
 'и',
 ' ',
 'развитая',
 ' ',
 'производственная',
 ' ',
 'инфраструктура',
 ' ',
 'резко',
 ' ',
 'отличалиет',
 ' ',
 'Армению',
 ' ',
 'от',
 ' ',
 'других',
 ' ',
 'регионов',
 ' ',
 'СССР',
 '',
 ',',
 ' ',
 'где',
 ' ',
 'доминировали',
 ' ',
 'добывающие',
 ' ',
 'отрасли',
 '',
 ',',
 ' ',
 'а',
 ' ',
 'экономика',
 ' ',
 'строилась',
 ' ',
 'на',
 ' ',
 'поставке',
 ' ',
 'сырьевых',
 ' ',
 'ресурсов',
 '',
 '.',
 ' ']

In [ ]:
conllu_path = '/content/drive/MyDrive/UD'
conv_path =  '/content/drive/MyDrive/UD_converted'

In [ ]:
for f in os.listdir(conllu_path):
    data_path = os.path.join(conllu_path, f)
    save_path = os.path.join(conv_path, f[:f.find('.')] + '.csv')
    convert_ud_file(file_path=data_path, save_path=save_path)


In [ ]:
a = os.listdir(conllu_path)[0]
os.path.join(conv_path, a[:a.find('.')] + '.csv')

'/content/drive/MyDrive/UD_converted/ru_syntagrus-ud-train-c.csv'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('Deeppavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
s = '''Я, видимо, сегодня не приду: "Баста".'''
s1 = '''Я , видимо , сегодня не приду : " Баста " .'''

In [ ]:
tokenizer.encode(s, add_special_tokens=False)

[839, 128, 29898, 128, 12904, 1699, 9987, 876, 156, 108, 30354, 626, 108, 132]

In [ ]:
tokenizer.encode(s1, add_special_tokens=False)

[839, 128, 29898, 128, 12904, 1699, 9987, 876, 156, 108, 30354, 626, 108, 132]

In [ ]:
sample = morph.parse('один')

In [ ]:
sample

[Parse(word='один', tag=OpencorporaTag('ADJF,Apro,Anum masc,sing,nomn'), normal_form='один', score=0.607142, methods_stack=((DictionaryAnalyzer(), 'один', 2293, 0),)),
 Parse(word='один', tag=OpencorporaTag('ADJF,Apro,Anum inan,masc,sing,accs'), normal_form='один', score=0.392857, methods_stack=((DictionaryAnalyzer(), 'один', 2293, 3),))]

In [ ]:
a.filter(id=lambda token: a[token['head']]['deprel']  == 'obj')

TypeError: 'int' object is not subscriptable

In [ ]:
url = "https://github.com/UniversalDependencies/UD_Russian-SynTagRus/blob/master/ru_syntagrus-ud-train-b.conllu"

In [ ]:
!wget --no-cache --backups=1 {url}

--2024-01-23 16:21:34--  https://github.com/UniversalDependencies/UD_Russian-SynTagRus/blob/master/ru_syntagrus-ud-train-b.conllu
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5417 (5.3K) [text/plain]
Failed to rename ru_syntagrus-ud-train-b.conllu to ru_syntagrus-ud-train-b.conllu.1: (2) No such file or directory
Saving to: ‘ru_syntagrus-ud-train-b.conllu’

ru_syntagrus-ud-tra 100%[===================>]   5.29K  --.-KB/s    in 0s      

2024-01-23 16:21:34 (101 MB/s) - ‘ru_syntagrus-ud-train-b.conllu’ saved [5417/5417]



In [ ]:
from minio import Minio
from tqdm import tqdm

from minio import Minio

START_WITH = 0  # строка начала
END = 20000000  # строка конца

files = [
    "fiction/Biblioteka_prikluchenij.txt.conllu"
]

def get_next_lines(minioClient, filename, pos_in_file, file_offset, pos):
    res_p = minioClient.get_object(
        "public", "syntax-parsed/" + filename, pos_in_file, file_offset
    ).data
    pos_in_file += file_offset
    # print(res_p)
    return res_p, pos_in_file

if __name__ == "__main__":
    file_offset = 100000
    path_dict_folder = "data"
    path_input = "syntax_parsed"

    minioClient = Minio(
        "minio.cosyco.ru:9000", access_key="public", secret_key="87654321", secure=False
    )
    # with open(os.path.join(conv_path, 'Biblioteka_prikluchenij_poly_agr.csv'), 'w') as f:
    #     f.write('')
    for filename in files:
        res_p, pos_in_file = get_next_lines(
            minioClient, filename, 0, file_offset, 0
        )

        pos = 0
        resid = b''
        last_h = res_p.rfind(35)
        # print('a', res_lines)
        # print(resid)
        res_lines, resid = resid + res_p[:last_h], res_p[last_h:]
        # print(res_lines)
        convert_ud_bytes(res_lines, conv_path)
        # break
        for i in tqdm(range(END)):
            # processing and changing pos.
            # print(pos, len(res_p))
            # if i > 10:
            #     break
            if True:

                if True:
                # try:
                    res_lines, pos_in_file = get_next_lines(
                        minioClient, filename, pos_in_file, file_offset, pos
                    )
                    last_h = res_lines.rfind(35)
                    # print('a', res_lines)
                    # print(resid)
                    res_lines, resid = resid + res_lines[:last_h], res_lines[last_h:]
                    # print(res_lines)
                    convert_ud_bytes(res_lines, conv_path)
                # except Exception:
                #     print(Exception)
                #     break
                # pos = 0

  0%|          | 23820/20000000 [1:54:38<1602:27:43,  3.46it/s]


S3Error: S3 operation failed; code: InvalidRange, message: The requested range is not satisfiable, resource: /public/syntax-parsed/fiction/Biblioteka_prikluchenij.txt.conllu, request_id: 17B8F69C5ABE50D2, host_id: 09af73c008d98bbaad13ba7a647ae7110ee0dcdf4b9aecdc6c7a653f14e9420a, bucket_name: public, object_name: syntax-parsed/fiction/Biblioteka_prikluchenij.txt.conllu

In [ ]:
data_s = res_lines.decode('utf-8')
data_iter = parse(data_s)
samples = []
for sample in data_iter:
    break


In [ ]:
data_iter[1]

TokenList<—, Эта, книга, рассказывает, о, народной, освободительной, войне, в, Индии, ., metadata={text: "— Эта книга рассказывает о народной освободительной войне в Индии."}>

In [ ]:
data_iter[1][-2]

{'id': 10,
 'form': 'Индии',
 'lemma': 'индия',
 'upos': 'PROPN',
 'xpos': None,
 'feats': {'Animacy': 'Inan',
  'Case': 'Loc',
  'Gender': 'Fem',
  'Number': 'Sing'},
 'head': 8,
 'deprel': 'nmod',
 'deps': None,
 'misc': None}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base = pd.read_csv('/content/drive/MyDrive/UD_converted/Biblioteka_prikluchenij_base.csv', names=['id', 'base'])

In [ ]:
agr = pd.read_csv('/content/drive/MyDrive/UD_converted/Biblioteka_prikluchenij_poly_agr.csv', names=['id', 'poly_agr'])

In [ ]:
base.shape

(2298483, 2)

In [ ]:
agr.columns

Index(['id', 'poly_agr'], dtype='object')

In [ ]:
base['poly_agr'] = agr['poly_agr']

In [ ]:
base['same'] = base['poly_agr'] == base['base']

In [ ]:
base[base['same']]

,id,base,poly_agr,same
0,0,« Опасный беглец » .,« Опасный беглец » .,True
1,1,— Эта книга рассказывает о народной освободите...,— Эта книга рассказывает о народной освободите...,True
2,2,В середине прошлого века угнетенные народы Инд...,В середине прошлого века угнетенные народы Инд...,True
4,4,Свидетелем страшной расправы голландских колон...,Свидетелем страшной расправы голландских колон...,True
5,5,Этот день явился решающим в его жизни .,Этот день явился решающим в его жизни .,True
...,...,...,...,...
2298478,61,- Лида !,- Лида !,True
2298479,62,Аркадий Михайлович !,Аркадий Михайлович !,True
2298480,63,Олекса Мартынович !,Олекса Мартынович !,True
2298481,64,Тарас ! - проговорил он тихим голосом в сильно...,Тарас ! - проговорил он тихим голосом в сильно...,True


In [ ]:
base.to_csv('/content/drive/MyDrive/UD_converted/Biblioteka_prikluchenij_bpa.csv')